<a href="https://colab.research.google.com/github/kishorepv/search/blob/main/%5BBUG_FIXED%5D_2_AI_Introduction_to_Lexical_and_BM25_TFIDF_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lexical tokenization - TF\*IDF

Let's walk through a basic introduction to lexical search.

### Who you are:

An ML engineer with enough comfort with Python data stack (pandas, numpy, etc) that wants to understand traditional search engines (ie Elasticsearch, etc)

### What this is

A run through of the core concepts behind lexical search.


## This notebook: TF\*IDF scoring

We [previously discussed controlling index and query time tokenization](https://colab.research.google.com/drive/1RGNkq4SOZMvlFvpHq3IKgNJdCTlHqiek). But we haven't even touched on how a "score" between a token and some tokenized text occurs. Now we can get into that.

In [1]:
!pip install searcharray

from searcharray import SearchArray
import pandas as pd
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 18.5 MB/s eta 0:00:00


## Tokenize and index

Last time we made a bit smarter tokenizer. Nothing too fancy, but interesting enough to make basic matching work

In [2]:
from string import punctuation


def better_tokenize(text):
    lowercased = text.lower()
    without_punctuation = lowercased.translate(str.maketrans('', '', punctuation))
    split = without_punctuation.split()
    return split


chat_transcript = [
  "Hi this is Doug, I have a complaint about the weather",
  "Doug, this is Tom, support for Earth's Climate, how can we help you doug?",
  "Tom, can I speak to your manager?",
  "Hi, this is Sue, Tom's boss. What can I do for you?",
  "I'd like to complain about the ski conditions in West Virginia",
  "Oh doug thats terrible, lets see what we can do."
]

msgs = pd.DataFrame({"name": ["Doug", "Tom", "Doug", "Sue", "Doug", "Sue"],
                     "msg": chat_transcript})
msgs['msg_tokenized'] = SearchArray.index(msgs['msg'],
                                          tokenizer=better_tokenize)
msgs

2025-09-21 05:31:45,692 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2025-09-21 05:31:45,695 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2025-09-21 05:31:45,697 - searcharray.indexing - INFO - Tokenizing 6 documents


...

2025-09-21 05:31:45,716 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2025-09-21 05:31:45,720 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete


,name,msg,msg_tokenized
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'a', 'weather', 'this', 'i', 'doug', 'a..."
1,Tom,"Doug, this is Tom, support for Earth's Climate...","Terms({'we', 'you', 'this', 'doug', 'for', 'cl..."
2,Doug,"Tom, can I speak to your manager?","Terms({'speak', 'i', 'tom', 'can', 'your', 'to..."
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'you', 'toms', 'do', 'this', 'sue', 'i'..."
4,Doug,I'd like to complain about the ski conditions ...,"Terms({'like', 'complain', 'the', 'about', 'sk..."
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'we', 'do', 'lets', 'doug', 'see', 'wha..."


## Search (again)

Recall we made an "AND" search, returning documents that match both tokens

In [3]:
QUERY = "doug complaint"
query_tokenized = better_tokenize(QUERY)
matches = np.zeros(len(msgs), dtype=np.bool)
for query_token in query_tokenized:
    matches |= (msgs['msg_tokenized'].array.score(query_token) > 0)

msgs[matches]

,name,msg,msg_tokenized
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'a', 'weather', 'this', 'i', 'doug', 'a..."
1,Tom,"Doug, this is Tom, support for Earth's Climate...","Terms({'we', 'you', 'this', 'doug', 'for', 'cl..."
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'we', 'do', 'lets', 'doug', 'see', 'wha..."


## Let's generate scores

SearchArray by default scores with BM25, but before we look at BM25, let's build some simpler ways of scoring to see how we can control this process.

Scoring is controlled with a `similarity`. Note, this isn't the same as something like `cosine similarity` in a vector search concept. It's a different kind of similarity: the "similarity" between a single individual search term and a field, as a function of several statistics (listed below).

In [4]:
from searcharray.similarity import Similarity

def term_counts(term_freqs: np.ndarray,        # TF array of every doc in the index
                doc_freqs: np.ndarray,         # Doc freq array of every term (> 1 if a phrase)
                doc_lens: np.ndarray,          # Every documents length (same shape as TF)
                avg_doc_lens: int,             # avg doc length of corpus
                num_docs: int) -> np.ndarray:     # total number of docs in corpus

    return term_freqs


In [5]:
msgs['msg_tokenized'].array.score('doug', similarity=term_counts)

array([1., 2., 0., 0., 0., 1.], dtype=float32)

In [6]:
QUERY = "doug complaint"
query_tokenized = better_tokenize(QUERY)

# ACCUMULATE SCORES
scores = np.zeros(len(msgs))
for query_token in query_tokenized:
    # PASS SIMILARITY
    score = msgs['msg_tokenized'].array.score(query_token,
                                              similarity=term_counts)
    print(f"Term '{query_token}' score: {score}")
    scores += score


msgs['scores'] = scores
msgs.sort_values('scores', ascending=False)

Term 'doug' score: [1. 2. 0. 0. 0. 1.]
Term 'complaint' score: [1. 0. 0. 0. 0. 0.]


,name,msg,msg_tokenized,scores
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'a', 'weather', 'this', 'i', 'doug', 'a...",2.0
1,Tom,"Doug, this is Tom, support for Earth's Climate...","Terms({'we', 'you', 'this', 'doug', 'for', 'cl...",2.0
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'we', 'do', 'lets', 'doug', 'see', 'wha...",1.0
2,Doug,"Tom, can I speak to your manager?","Terms({'speak', 'i', 'tom', 'can', 'your', 'to...",0.0
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'you', 'toms', 'do', 'this', 'sue', 'i'...",0.0
4,Doug,I'd like to complain about the ski conditions ...,"Terms({'like', 'complain', 'the', 'about', 'sk...",0.0


## Notice - we're not preferring any particular term

We search for `doug`, `complaint`, but with raw term counts, two occurences of `doug` matter just as much as `complaint`. One solution is to begin considering / preferring the rare terms in scoring. That's where we divide TF by the DF (or document frequency - how many documents a term occurs in)

In [7]:
from searcharray.similarity import Similarity

def tf_over_df(term_freqs: np.ndarray,        # TF array of every doc in the index
               doc_freqs: np.ndarray,         # Doc freq array of every term (> 1 if a phrase)
               doc_lens: np.ndarray,          # Every documents length (same shape as TF)
               avg_doc_lens: int,             # avg doc length of corpus
               num_docs: int) -> np.ndarray:     # total number of docs in corpus

    return term_freqs / doc_freqs


In [8]:
QUERY = "doug complaint"
query_tokenized = better_tokenize(QUERY)

# ACCUMULATE SCORES
scores = np.zeros(len(msgs))
for query_token in query_tokenized:
    # PASS SIMILARITY
    score = msgs['msg_tokenized'].array.score(query_token,
                                              similarity=tf_over_df)
    print(f"Term '{query_token}' score: {score}")
    scores += score


msgs['scores'] = scores
msgs.sort_values('scores', ascending=False)

Term 'doug' score: [0.33333333 0.66666667 0.         0.         0.         0.33333333]
Term 'complaint' score: [1. 0. 0. 0. 0. 0.]


,name,msg,msg_tokenized,scores
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'a', 'weather', 'this', 'i', 'doug', 'a...",1.333333
1,Tom,"Doug, this is Tom, support for Earth's Climate...","Terms({'we', 'you', 'this', 'doug', 'for', 'cl...",0.666667
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'we', 'do', 'lets', 'doug', 'see', 'wha...",0.333333
2,Doug,"Tom, can I speak to your manager?","Terms({'speak', 'i', 'tom', 'can', 'your', 'to...",0.000000
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'you', 'toms', 'do', 'this', 'sue', 'i'...",0.000000
4,Doug,I'd like to complain about the ski conditions ...,"Terms({'like', 'complain', 'the', 'about', 'sk...",0.000000


## TF divided by DF == TF * IDF ~= BM25

IDF just means "Inverse document frequency", i.e. 1/document frequency. And TF * 1 / document frequency == TF / DF.

**BM25 is a highly tuned version of this** we'll see it later.

## Breadcrumbs for Elasticsearch, Vespa etc

A good search engine lets you control text matching scoring. Elasticsearch / Lucene have a pluggable [concept of a similarity](https://www.elastic.co/docs/reference/elasticsearch/index-settings/similarity). Vespa gives you raw control of computation using term statistics